<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [ ]:
import pandas as pd
import psycopg2

In [ ]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [ ]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [ ]:
# текст запроса
query_3_1 = f'''select count(*)
            from public.vacancies
'''

In [ ]:
pd.read_sql_query(query_3_1, connection)

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [ ]:
count_employers = f''' select count(*)
           from public.employers
        '''

In [ ]:
pd.read_sql_query(count_employers, connection)

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [ ]:
area_count = f''' select count(*)
        from public.areas
'''

In [ ]:
pd.read_sql_query(area_count, connection)

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [ ]:
industry_count = f''' select count(*)
            from public.industries
'''

In [ ]:
pd.read_sql_query(industry_count, connection)

***

In [ ]:
# выводы по предварительному анализу данных

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [ ]:
top_five_cities = f''' select
    ar.name,
    count(ar.name)
from public.areas as ar,
     public.vacancies as va
where ar.id = va.area_id
group by ar.name 
order by count(ar.name) DESC
LIMIT 5
'''

In [ ]:
pd.read_sql_query(top_five_cities, connection)

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [ ]:
salary_not_empty = f'''select count(*)
from public.vacancies
where salary_from > 0 
    or salary_to > 0
'''

In [ ]:
pd.read_sql_query(salary_not_empty, connection)

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [ ]:
average_salary = f'''select 
    avg(salary_from),
    avg(salary_to)
from public.vacancies
'''

In [ ]:
pd.read_sql_query(average_salary, connection)

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [ ]:
schedule = f'''select
    schedule,
    employment,
    count(schedule)
from public.vacancies
group by schedule, employment 
order by count(schedule) DESC
LIMIT 5
'''

In [ ]:
pd.read_sql_query(schedule, connection)

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [ ]:
employment_order = f'''select
    experience,
    count(experience)
from public.vacancies
group by experience
order by count(experience) ASC
'''

In [ ]:
pd.read_sql_query(employment_order, connection)

***

In [ ]:
# выводы по детальному анализу вакансий

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
top_5 = f'''select emp.name,
       count(emp.name)
from public.vacancies as vac,
    public.employers as emp
where vac.employer_id = emp.id
group by emp.name
order by count(emp.name) DESC
LIMIT 5 '''

In [ ]:
pd.read_sql_query(top_5, connection)

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
regions = f'''select 
    ar.name,
    emp.area,
    count(emp.area)
from public.vacancies as vac
full outer join public.employers emp on vac.area_id = emp.area
full outer join public.areas as ar on vac.area_id = ar.id
where vac.area_id is null 
    and emp.name is not null 
group by ar.name, emp.area
order by count(emp.area) DESC'''


In [ ]:
pd.read_sql_query(regions, connection)

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
top_company = f'''select 
       employer_id,
       count(distinct area_id)
from vacancies
group by employer_id
order by count(distinct area_id) DESC
LiMIT 1'''

In [ ]:
pd.read_sql_query(top_company, connection)

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
difference = f'''select count(distinct id) - count(distinct employer_id)
from employers_industries,
     employers'''


In [ ]:
pd.read_sql_query(difference, connection)

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
number_three =f'''select  
        ei.employer_id,
       e.name,
       count(distinct industry_id)
from employers_industries ei join employers e on ei.employer_id = e.id
group by ei.employer_id, e.name
having count(distinct industry_id) = 4 
order by count(distinct industry_id) DESC, e.name
LIMIT 3'''

In [ ]:
pd.read_sql_query(number_three, connection)

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
software_company = f'''select count(*)
from employers_industries ei join employers e on ei.employer_id = e.id
                             join industries i on i.id = ei.industry_id
where i.name = 'Разработка программного обеспечения'
'''

In [ ]:
pd.read_sql_query(software_company, connection)

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [ ]:
# код для получения списка городов-милионников

In [ ]:
# текст запроса

In [ ]:
# результат запроса

***

In [ ]:
# выводы по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса

In [ ]:
# результат запроса

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
selection = f'''select   
    count(*)
from vacancies
where (name ~* 'data scien(ce|tist)|исследователь данных|ML|machine learning|машинн.*обучен.*' and name !~* 'HTML') 
   and (name ~* 'junior' or experience = 'Нет опыта' or employment = 'Стажировка') '''

In [ ]:
pd.read_sql_query(selection, connection)

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
Sql_selection = f'''select   
    count(*)
from vacancies
where ((name ilike '%data science%') or (name ilike  '%data scientist%') or (name ilike '%исследователь данных%') or (name ilike '%machine learning%') or (name ilike '%машинн%обучен%') 
     or (name not like '%HTML%' and name like '%ML%'))
     and ((key_skills ilike '%postgre%s%') or (key_skills ilike '%SQL%'))'''

In [ ]:
pd.read_sql_query(Sql_selection, connection) 

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
Python_selection = f'''select   
    count(*)
from vacancies
where ((name ilike '%data science%') or (name ilike  '%data scientist%') or (name ilike '%исследователь данных%') or (name ilike '%machine learning%') or (name ilike '%машинн%обучен%') 
     or (name not like '%HTML%' and name like '%ML%'))
     and (key_skills ilike '%Python%')'''

In [ ]:
pd.read_sql_query(Python_selection, connection) 

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
Length = f'''select avg(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), ''))+1)
from vacancies 
where ((name ilike '%data science%') or (name ilike  '%data scientist%') or (name ilike '%исследователь данных%') or (name ilike '%machine learning%') or (name ilike '%машинн%обучен%') 
     or (name not like '%HTML%' and name like '%ML%'))
     and (key_skills is not Null)'''

In [ ]:
pd.read_sql_query(Length, connection)

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

***

In [ ]:
# выводы по предметному анализу

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования